![](q6-10.png)

In [2]:
from datasets import load_dataset
from tqdm import tqdm
import random

en = load_dataset('facebook/voxpopuli', 'en', split='train', streaming=True).take(200)
es = load_dataset('facebook/voxpopuli', 'es', split='train', streaming=True).take(200)
ro = load_dataset('facebook/voxpopuli', 'ro', split='train', streaming=True).take(200)

subset = []
for sample in tqdm(en):
    subset.append(sample)
for sample in tqdm(es):
    subset.append(sample)
for sample in tqdm(ro):
    subset.append(sample)

random.seed(10)
random.shuffle(subset)

200it [00:20,  9.82it/s]
200it [00:13, 14.93it/s]
200it [00:14, 13.87it/s]


In [21]:
from transformers import Wav2Vec2Model, Wav2Vec2Processor

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h", output_hidden_states=True)
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import torch

def get_encoding(sample):
    enc = processor(sample['audio']['array'], sampling_rate=16_000, padding=True, return_tensors='pt')
    with torch.no_grad():
        out = wav_model(**enc)
    return out

In [23]:
out = get_encoding(subset[0])

In [76]:
out.last_hidden_state.squeeze(0).mean(axis=0).unsqueeze(0).shape

torch.Size([1, 768])

In [33]:
from torch import nn

In [99]:
class CNN(nn.Module):

    def __init__(self, input_dim, num_classes, input_length=768):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * (input_length // 9), 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool1d(x, kernel_size=3)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool1d(x, kernel_size=3)
        x = x.view(x.size(0), -1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        
cnn = CNN(1, 3)
n_params = 0
for p in cnn.parameters():
    print(p.numel())
    print('---')
    n_params += p.numel()
n_params

192
---
64
---
24576
---
128
---
1392640
---
128
---
384
---
3
---


1418115

In [100]:
cnn(x.unsqueeze(0).unsqueeze(1))

tensor([[ 0.0165,  0.0859, -0.0368]], grad_fn=<AddmmBackward0>)